In [ ]:
import pandas as pd
import utils
import plotly.graph_objects as go

import numpy as np
#import matplotlib.pyplot as plt
import pandas_datareader as web
import pprint

In [ ]:
pair = "GBP_USD"
granularity = "M5"

In [ ]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [ ]:
#df

In [ ]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [ ]:
df_wt = df[['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'ask_h', 'ask_l','ask_c', 'bid_h', 'bid_l', 'bid_c']].copy()

In [ ]:
df_wt.tail()

,time,volume,mid_o,mid_h,mid_l,mid_c,ask_h,ask_l,ask_c,bid_h,bid_l,bid_c
629,2021-11-30 23:35:00+00:00,46,0.68282,0.68290,0.68274,0.68280,0.68300,0.68285,0.68290,0.68280,0.68263,0.68271
630,2021-11-30 23:40:00+00:00,57,0.68282,0.68293,0.68253,0.68256,0.68302,0.68263,0.68267,0.68284,0.68241,0.68246
631,2021-11-30 23:45:00+00:00,60,0.68254,0.68258,0.68205,0.68210,0.68268,0.68217,0.68220,0.68248,0.68193,0.68201
632,2021-11-30 23:50:00+00:00,55,0.68208,0.68227,0.68202,0.68227,0.68237,0.68213,0.68237,0.68217,0.68192,0.68217
633,2021-11-30 23:55:00+00:00,63,0.68229,0.68268,0.68229,0.68242,0.68279,0.68237,0.68250,0.68258,0.68221,0.68234


In [ ]:
df_wt['ap'] = (df_wt['mid_h'] + df_wt['mid_l'] + df_wt['mid_c'])/3

#df_wt.dropna(inplace=True)


def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

n1=10
n2=21
#df_wt['esa'] = calculate_ema(df_wt['ap'], 10) # Add this line to save EMA values in a list
df_wt['esa'] = df_wt['ap'].ewm(span=n1, min_periods=n1).mean()


df_wt['d'] = (abs(df_wt['ap'] - df_wt['esa'])).ewm(span=n1, min_periods=n1).mean()
df_wt['ci'] = (df_wt['ap'] - df_wt['esa']) / (0.015 * df_wt['d'])
df_wt['tci'] = df_wt['ci'].ewm(span=n2, min_periods=n2).mean()
 
df_wt['wt1'] = df_wt['tci']
#df_wt['wt2'] = df_wt['wt1'].sma(window = 4).mean
df_wt['wt2'] = df_wt.wt1.rolling(window=4).mean()


df_wt['EMA_9'] = df_wt.mid_c.ewm(span=9).mean()
#df_wt['EMA_9'] = df_wt.mid_c.rolling(window = 9).ewm().mean()
'''if (df_wt.mid_c.rolling(window=15).max()) > (df_wt.mid_o.rolling(window=15).max()):
    df_wt['high'] = df_wt.mid_c.rolling(window=15).max()
else:
    df_wt['high'] = df_wt.mid_o.rolling(window=15).max()
if (df_wt.mid_o.rolling(window=15).max()) < (df_wt.mid_c.rolling(window=15).max()):
    df_wt['low'] = df_wt.mid_o.rolling(window=15).max()
else:
    df_wt['low'] = df_wt.mid_c.rolling(window=15).max()
'''
df_wt['high'] = df_wt.mid_h.rolling(window=15).max()

df_wt['low'] = df_wt.mid_l.rolling(window=15).min()


In [ ]:
def is_trade(row):
    if row.wt2 > 0 and row.wt2_prev < 0 and row.mid_h >= row.high and (row.mid_c-row.mid_o)>0 and (((row.mid_h-row.mid_c)/row.mid_c)*1.3) < (((row.mid_o-row.mid_l)/row.mid_o)*1.0) and row.IS_TRADE_prev == 0:
        return 1
    elif row.wt2 < 0 and row.wt2_prev > 0 and row.mid_l <= row.low and (row.mid_c-row.mid_o)<0 and (((row.mid_h-row.mid_c)/row.mid_c)*1.0) > (((row.mid_o-row.mid_l)/row.mid_o)*1.3) and row.IS_TRADE_prev == 0:
        return -1
    elif row.wt2 > 13.1217 and row.wt2_prev < 13.1217 and row.mid_h >= row.high and (row.mid_c-row.mid_o)>0 and (((row.mid_h-row.mid_c)/row.mid_c)*1.3) < (((row.mid_o-row.mid_l)/row.mid_o)*1.0) and row.IS_TRADE_prev == 0:
        return 1
    elif row.wt2 < 13.1217 and row.wt2_prev > 13.1217 and row.mid_l <= row.low and (row.mid_c-row.mid_o)<0 and (((row.mid_h-row.mid_c)/row.mid_c)*1.0) > (((row.mid_o-row.mid_l)/row.mid_o)*1.3) and row.IS_TRADE_prev == 0:
        return -1
    #return 0

def open_trade(row):
    return 0

In [ ]:
'''
df_wt['wt2_prev'] = df_wt.wt2.shift(1)

df_wt['DIFF'] = df_wt.wt1 - df_wt.wt2
df_wt['IS_TRADE_prev'] = 0
df_wt['IS_TRADE'] = df_wt.apply(is_trade, axis=1)
df_wt['IS_TRADE_prev'] = df_wt.IS_TRADE.shift(1)
df_wt['IS_TRADE'] = df_wt.apply(is_trade, axis=1)

df_trades_extract = df_wt[df_wt.IS_TRADE != 0].copy()
df_trades = df_trades_extract[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'wt1', 'wt2', 'high', 'low', 'wt2_prev', 'DIFF', 'IS_TRADE', 'IS_TRADE_prev']].copy()
'''

"\ndf_wt['wt2_prev'] = df_wt.wt2.shift(1)\n\ndf_wt['DIFF'] = df_wt.wt1 - df_wt.wt2\ndf_wt['IS_TRADE_prev'] = 0\ndf_wt['IS_TRADE'] = df_wt.apply(is_trade, axis=1)\ndf_wt['IS_TRADE_prev'] = df_wt.IS_TRADE.shift(1)\ndf_wt['IS_TRADE'] = df_wt.apply(is_trade, axis=1)\n\ndf_trades_extract = df_wt[df_wt.IS_TRADE != 0].copy()\ndf_trades = df_trades_extract[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'wt1', 'wt2', 'high', 'low', 'wt2_prev', 'DIFF', 'IS_TRADE', 'IS_TRADE_prev']].copy()\n"

In [ ]:
df_wt['wt2_prev'] = df_wt.wt2.shift(1)

df_wt['DIFF'] = df_wt.wt1 - df_wt.wt2
df_wt['IS_TRADE_prev'] = 0
df_wt['IS_TRADE'] = df_wt.apply(is_trade, axis=1)
df_wt['IS_TRADE_prev'] = df_wt.IS_TRADE.shift(1).fillna(0).astype(int)
df_wt['IS_TRADE'] = df_wt.apply(is_trade, axis=1)
df_wt.dropna(inplace=True)

df_trades_extract = df_wt[df_wt.IS_TRADE != 0].copy()
df_trades = df_trades_extract[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'wt1', 'wt2', 'high', 'low', 'wt2_prev', 'DIFF', 'IS_TRADE', 'IS_TRADE_prev']].copy()
#df_trades = df_wt[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'wt1', 'wt2', 'high', 'low', 'wt2_prev', 'DIFF', 'IS_TRADE', 'IS_TRADE_prev']].copy()
#above line creates df with non trades as well

In [ ]:
#df_trades
#df_wt

In [ ]:
def get_stop_loss(row):
    if row.IS_TRADE == 1:
        return (row.mid_c * 0.9995)
    elif row.IS_TRADE == -1:
        return (row.mid_c * 1.0005)
    else:
        return 0


def get_take_profit(row):
    if row.IS_TRADE == 1:
        return (row.mid_c * 1.0005)
    elif row.IS_TRADE == -1:
        return (row.mid_c * 0.9995)
    else:
        return 0

def get_entry_stop(row):
    if row.IS_TRADE == 1:
        return row.ask_c
    elif row.IS_TRADE == -1:
        return row.bid_c
    else:
        return 0


In [ ]:
df_wt['ENTRY'] = df_wt.apply(get_entry_stop, axis=1)
df_wt['STOPLOSS'] = df_wt.apply(get_stop_loss, axis=1)
df_wt['TAKEPROFIT'] = df_wt.apply(get_take_profit, axis=1)

In [ ]:
class Trade():
    def __init__(self, row):
        self.candle_date = row.time
        self.direction = row.IS_TRADE
        self.entry = row.ENTRY
        self.exit = None
        self.TP = row.TAKEPROFIT
        self.SL = row.STOPLOSS
        self.running = False
        self.result = None
        self.stopped = None
        self.breakeven = False
        

    def update(self, row):
        if self.running == True:
            self.update_result(row)
        else:
            self.check_entry(row)    
    
    def check_entry(self, row):
        if self.direction == 1  or self.direction == -1:
            self.index = row.name
            self.opened = row.time
            self.running = True

    def update_result(self, row):
        if self.direction == 1:
            if (row.bid_h-self.entry)!=0 and self.entry!=0:
                if (((row.bid_h-self.entry)/self.entry)*100) >= .25 :
                    self.result = ( (self.TP-self.entry)/self.entry )*100
            elif row.bid_h >= self.TP:
                row.IS_TRADE = self.direction
                self.TP = row.TAKEPROFIT
            elif row.bid_l <= self.SL:
                self.result = ( (self.SL-self.entry)/self.entry )*100
                
            elif row.bid_c <= self.entry and self.breakeven == True:
                self.result = 0.0
            elif self.entry!=0 and (( (row.bid_c-self.entry)/self.entry )*100) >= 0.05:
                #self.breakeven = False
                self.breakeven = True

        elif self.direction == -1:
            if (row.ask_l-self.entry)!=0 and self.entry!=0:
                if (((row.ask_l-self.entry)/self.entry)*100) <= -.25 :
                    self.result = -((self.TP-self.entry)/self.entry)*100
            if row.ask_l <= self.TP:
                row.IS_TRADE = self.direction
                self.TP = row.TAKEPROFIT
            elif row.ask_h >= self.SL and self.entry!=0:
                self.result = ( (self.entry-self.SL)/self.entry )*100

            elif row.ask_c >= self.entry and self.breakeven == True:
                self.result = 0.0
            elif self.entry!=0 and (((row.ask_c-self.entry)/self.entry)*100) <= -0.05:
                #self.breakeven = False
                self.breakeven = True
        '''else:
            print ("direction is 0 error")'''

        if self.result is not None:
            self.running = False
            self.stopped = row.time
            if self.direction == 1:
                self.exit = row.bid_c
            elif self.direction == -1:
                self.exit = row.ask_c



In [ ]:
df_wt.reset_index(inplace=True)

open_trades = []
closed_trades = []

for index, row in df_wt.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.stopped is not None:
            closed_trades.append(ot)
    
    open_trades = [x for x in open_trades if x.stopped is None]

    if row.IS_TRADE != 0:
        open_trades = [x for x in open_trades if x.running == True]
        open_trades.append(Trade(row))

def account_grow():
    return 0

ZeroDivisionError: float division by zero

In [ ]:
df_trades = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [ ]:
df_trades.reset_index(inplace=True)
df_trades["duration"] = df_trades["stopped"] - df_trades["opened"]
df_trades["cum_duration"] = df_trades["duration"].cumsum()
df_trades["avg_duration"] = df_trades["cum_duration"]/(df_trades.index + 1)

df_trades.head()

,level_0,candle_date,direction,entry,exit,TP,SL,running,result,stopped,breakeven,index,opened,duration,cum_duration,avg_duration
0,0,2021-01-04 23:40:00+00:00,-1,1.35630,1.35721,1.355722,1.357078,False,-0.057377,2021-01-04 23:55:00+00:00,False,267,2021-01-04 23:45:00+00:00,0 days 00:10:00,0 days 00:10:00,0 days 00:10:00
1,1,2021-01-05 02:25:00+00:00,1,1.35798,1.36126,1.358579,1.357221,False,0.044106,2021-01-05 16:20:00+00:00,False,300,2021-01-05 02:30:00+00:00,0 days 13:50:00,0 days 14:00:00,0 days 07:00:00
2,2,2021-01-06 01:30:00+00:00,-1,1.36137,1.36188,0.000000,1.362151,False,0.000000,2021-01-06 04:20:00+00:00,True,575,2021-01-06 01:35:00+00:00,0 days 02:45:00,0 days 16:45:00,0 days 05:35:00
3,3,2021-01-05 19:25:00+00:00,1,1.36342,1.36656,1.364022,1.362658,False,0.044129,2021-01-06 10:45:00+00:00,False,504,2021-01-05 19:30:00+00:00,0 days 15:15:00,1 days 08:00:00,0 days 08:00:00
4,4,2021-01-06 11:55:00+00:00,-1,1.36476,1.36115,0.000000,1.365522,False,100.000000,2021-01-06 12:45:00+00:00,True,700,2021-01-06 12:00:00+00:00,0 days 00:45:00,1 days 08:45:00,0 days 06:33:00


In [ ]:
df_trades.tail()

,level_0,candle_date,direction,entry,exit,TP,SL,running,result,stopped,breakeven,index,opened,duration,cum_duration,avg_duration
350,350,2021-11-29 03:45:00+00:00,-1,1.33313,1.33366,1.332573,1.333907,False,-0.058255,2021-11-29 05:30:00+00:00,False,67325,2021-11-29 03:50:00+00:00,0 days 01:40:00,999 days 23:40:00,2 days 20:22:30.427350427
351,351,2021-11-29 18:05:00+00:00,1,1.33083,1.33409,1.331425,1.330095,False,0.044737,2021-11-30 08:35:00+00:00,False,67497,2021-11-29 18:10:00+00:00,0 days 14:25:00,1000 days 14:05:00,2 days 20:13:18.579545454
352,352,2021-11-30 00:50:00+00:00,1,1.33232,1.33573,1.332886,1.331554,False,0.042491,2021-11-30 09:10:00+00:00,False,67576,2021-11-30 00:55:00+00:00,0 days 08:15:00,1000 days 22:20:00,2 days 20:03:06.968838526
353,353,2021-11-24 19:40:00+00:00,1,1.33306,1.33610,1.333646,1.332314,False,0.043996,2021-11-30 09:25:00+00:00,False,66659,2021-11-24 19:45:00+00:00,5 days 13:40:00,1006 days 12:00:00,2 days 20:14:14.237288135
354,354,2021-11-25 18:05:00+00:00,1,1.33324,1.33643,1.333727,1.332393,False,0.036492,2021-11-30 09:30:00+00:00,False,66924,2021-11-25 18:10:00+00:00,4 days 15:20:00,1011 days 03:20:00,2 days 20:21:31.267605633


In [ ]:
'''list_wrong = [(x,y) for (x,y) in df_trades.iterrows() if y.result>1]
df_wrong = pd.DataFrame(np.array(list_wrong).reshape(,3), columns = list("abc"))
'''
def wrong_find(row):
    if row.result>1:
        print(row)
    elif row.result<-1:
        print(row)

#df_wrong = 
df_trades.apply(wrong_find, axis=1)
#print(df_wrong)

level_0                                 4
candle_date     2021-01-06 11:55:00+00:00
direction                              -1
entry                             1.36476
exit                              1.36115
TP                                    0.0
SL                               1.365522
running                             False
result                              100.0
stopped         2021-01-06 12:45:00+00:00
breakeven                            True
index                                 700
opened          2021-01-06 12:00:00+00:00
duration                  0 days 00:45:00
cum_duration              1 days 08:45:00
avg_duration              0 days 06:33:00
Name: 4, dtype: object
level_0                                 6
candle_date     2021-01-07 14:30:00+00:00
direction                              -1
entry                               1.358
exit                              1.35498
TP                                    0.0
SL                               1.358749
running    

0      None
1      None
2      None
3      None
4      None
       ... 
350    None
351    None
352    None
353    None
354    None
Length: 355, dtype: object

In [ ]:
def win_find(row):
    if row.result>0.1 and row.result<100:
        print(row)
    elif row.result<-0.1 and row.result<100:
        print(row)
df_trades.apply(win_find, axis=1)


0      None
1      None
2      None
3      None
4      None
       ... 
350    None
351    None
352    None
353    None
354    None
Length: 355, dtype: object

In [ ]:
leverage = 20.0
df_trades.result.sum()*leverage
#incorporate leverage directly into compounding of P/L

36074.98117981033

In [ ]:
df_trades.result.max()

100.0

In [ ]:
df_trades.index

RangeIndex(start=0, stop=355, step=1)

In [ ]:
df_wt.tail()

,index,time,volume,mid_o,mid_h,mid_l,mid_c,ask_h,ask_l,ask_c,...,EMA_9,high,low,wt2_prev,DIFF,IS_TRADE_prev,IS_TRADE,ENTRY,STOPLOSS,TAKEPROFIT
67848,628,2021-11-30 23:35:00+00:00,78,1.33104,1.33111,1.33052,1.33054,1.33123,1.33064,1.33065,...,1.330639,1.33146,1.32959,35.366116,0.349047,0,0,0.0,0.0,0.0
67849,629,2021-11-30 23:40:00+00:00,93,1.33058,1.33079,1.33022,1.33033,1.33091,1.33033,1.33043,...,1.330577,1.33146,1.32959,39.433529,-5.389855,0,0,0.0,0.0,0.0
67850,630,2021-11-30 23:45:00+00:00,119,1.33031,1.33031,1.32974,1.32976,1.33044,1.32984,1.32988,...,1.330414,1.33146,1.32959,38.646241,-13.369083,0,0,0.0,0.0,0.0
67851,631,2021-11-30 23:50:00+00:00,43,1.32978,1.32989,1.32968,1.32971,1.33001,1.32978,1.32981,...,1.330273,1.33146,1.32959,33.625972,-16.656615,0,0,0.0,0.0,0.0
67852,632,2021-11-30 23:55:00+00:00,44,1.32974,1.32984,1.32950,1.32952,1.32995,1.32959,1.32963,...,1.330122,1.33146,1.32950,25.546412,-16.096288,0,0,0.0,0.0,0.0


In [ ]:
#df_wt.dropna(inplace=True)
df_wt.head(5)

,index,time,volume,mid_o,mid_h,mid_l,mid_c,ask_h,ask_l,ask_c,...,EMA_9,high,low,wt2_prev,DIFF,IS_TRADE_prev,IS_TRADE,ENTRY,STOPLOSS,TAKEPROFIT
0,42,2021-01-04 01:30:00+00:00,225,1.36825,1.36844,1.36779,1.36779,1.36855,1.36788,1.36788,...,1.367716,1.36844,1.36598,30.921038,5.636205,0,0,0.0,0.0,0.0
1,43,2021-01-04 01:35:00+00:00,151,1.36782,1.36819,1.36770,1.36819,1.36830,1.36780,1.36828,...,1.367811,1.36844,1.36598,35.047462,3.135158,0,0,0.0,0.0,0.0
2,44,2021-01-04 01:40:00+00:00,242,1.36822,1.36900,1.36822,1.36870,1.36909,1.36832,1.36881,...,1.367989,1.36900,1.36598,38.562345,5.993832,0,0,0.0,0.0,0.0
3,45,2021-01-04 01:45:00+00:00,197,1.36872,1.36880,1.36824,1.36876,1.36891,1.36833,1.36884,...,1.368143,1.36900,1.36598,42.459110,6.033690,0,0,0.0,0.0,0.0
4,46,2021-01-04 01:50:00+00:00,154,1.36878,1.36898,1.36822,1.36836,1.36907,1.36833,1.36846,...,1.368186,1.36900,1.36642,45.622601,3.804040,0,0,0.0,0.0,0.0


In [1]:
#df_wt.iloc[400:450]

In [ ]:
df_plot = df_wt.iloc[-100:].copy()

In [ ]:
df_plot = df_trades.iloc[200:300]
df_buys = df_plot[df_plot.direction == 1]
df_sells = df_plot[df_plot.IS_TRADE == -1]

AttributeError: 'DataFrame' object has no attribute 'IS_TRADE'

In [ ]:
df_plot.shape

(0, 17)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
fig.add_trace(go.Scatter(x=df_plot.time, 
    y=df_plot.MA_8,
    line=dict(color="#027FC3", width=2),
    line_shape='spline',
    name='MA_8'
    ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()

AttributeError: 'DataFrame' object has no attribute 'MA_8'